In [273]:
# read my pickle
# data = pickle.load(open("individuals_friendships_utilities.p","rb"))
# characteristics, connectivity, utilities = data

g_list = pickle.load(open("g_list.pkl", "rb"))
X_list = pickle.load(open("X_list.pkl", "rb"))

In [274]:
connectivity = g_list[1]
print(connectivity.shape)
# print(connectivity)
characteristics = X_list[1]
charac_short = characteristics[:,0:3]
print(charac_short.shape)
# print(charac_short)

(109, 109)
(109, 3)


In [276]:
# analyse_network(connectivity, characteristics)

In [68]:
small = np.array([[0,1,1,0],
                 [1,0,1,0],
                 [0,0,0,0],
                 [0,0,0,0]])
print(small)
print(np.sum(small))

charac_mini = charac_short[0:4]
print(charac_mini)

[[0 1 1 0]
 [1 0 1 0]
 [0 0 0 0]
 [0 0 0 0]]
4
[[ 2  1 10]
 [ 1  1  7]
 [ 2  1 12]
 [ 1  1  9]]


In [69]:
small5 = np.array([[0,1,1,1,1],
                 [1,0,1,1,0],
                 [1,0,0,0,0],
                 [1,0,0,0,0],
                  [1,0,1,1,0]])
print(small5)
print(np.sum(small5))

charac_mini5 = charac_short[0:5]
print(charac_mini5)

[[0 1 1 1 1]
 [1 0 1 1 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 1 1 0]]
12
[[ 2  1 10]
 [ 1  1  7]
 [ 2  1 12]
 [ 1  1  9]
 [ 1  1 11]]


In [253]:
analyse_network(small5, charac_mini5)

/Applications/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:159: UserWarning: Just one group: no segregation index for trait racepossible.


(2.4,
 1.25,
 0.24,
 0.5,
 0.23809523809523808,
 [0.5555555555555555, 0.15713484026367722],
 {'grade_10': 0.0,
  'grade_11': 0.0,
  'grade_12': 0.0,
  'grade_7': 0.0,
  'grade_9': 0.0,
  'race_1': 1.0,
  'sex_1': 0.2857142857142857,
  'sex_2': 0.4},
 [-0.10344827586206896, nan, 0.0])

In [12]:
# output:
# den_max_reach / density at maximum reach (RCHDEN): density of the reachability network 
# rel_den / relative density (RELDEN): amount of nodes divided by the maximum possible number of nodes
# p_symm_dyads / proportion symmetric dyads (PTCMUT): proportion of nodes that are reciprocal (counting once)
# mutuality_index / mutuality index (RHO2): Katz and Powell’s (1955) mutuality index. Measures the tendency for actors in a group to reciprocate choices.
den_max_reach, rel_den, p_symm_dyads, mutuality_index, clustering_coefficient = analyse_network(small)

In [270]:
import pickle
import numpy as np
import copy
import warnings

def analyse_network(connectivity, characteristics):
    # OUTPUT:
    # average degree of nodes network            > av_degree
    # density at maximum reach (RCHDEN)          > den_max_reach
    # relative density (RELDEN)                  > rel_den
    # proportion symmetric dyads (PTCMUT)        > p_symm_dyads
    # mutuality index (RHO2)                     > mutuality_index
    # amount of clustering (groups of friends)   > clustering_coefficient  [mean, std dev]
    # homophily index per trait in dict          > homoph_ind              {"0_", "1_", "2_"}
    # segregation index per trait                > segreg_ind              [sex, race, grade]
    
    # INPUT: 
    # - connectivity matrix with row-students nominating column-students as friends
    # - characteristics matrix with row per student, with integers indicating every group for each characteristic (sex, race, grade)
    
    nodes = connectivity.shape[0]
    mutual_d = 0
    asym_d = 0
#     trait = ["sex", "race", "grade", "school", "_", "_"]
    
    
    #density DENX2
    #DENX2 = np.sum(connectivity)/(nodes*(nodes-1))
    
    
    # density at maximum reach RCHDEN
    
    # define the function to tranfer adjacency matrix to reachability matrix  
    # Prints reachability matrix of graph[][] using Floyd Warshall algorithm 
    # function found on https://www.geeksforgeeks.org/transitive-closure-of-a-graph/
    reachability = copy.deepcopy(connectivity)
    '''reach[][] will be the output matrix that will finally 
    have reachability values. 
    Initialize the solution matrix same as input graph matrix'''
    reach =[i[:] for i in reachability] 
    '''Add all vertices one by one to the set of intermediate 
    vertices. 
    ---> Before start of a iteration, we have reachability value 
    for all pairs of vertices such that the reachability values 
    consider only the vertices in set  
    {0, 1, 2, .. k-1} as intermediate vertices. 
    ----> After the end of an iteration, vertex no. k is 
    added to the set of intermediate vertices and the  
    set becomes {0, 1, 2, .. k}'''
    for k in range(nodes): 
              
        # Pick all vertices as source one by one 
        for i in range(nodes): 
                  
            # Pick all vertices as destination for the 
            # above picked source 
            for j in range(nodes): 
                      
                # If vertex k is on a path from i to j,  
                    # then make sure that the value of reach[i][j] is 1 
                reach[i][j] = reach[i][j] or (reach[i][k] and reach[k][j]) 
    
    RCHDEN = np.sum(reach)/(nodes*(nodes-1))
    
    
    # relative density RELDEN
    RELDEN = np.sum(connectivity)/(10*nodes)


    # create upper triangular matrix with 2's on mutual dyads, 1's on asymmetric dyads and count occurrence
    added_up = np.triu(connectivity + np.transpose(connectivity))
    mutual_d = np.count_nonzero(added_up == 2)
    asym_d = np.count_nonzero(added_up == 1)
    total_d = mutual_d + asym_d
    
    # calculate proportion symmetric dyads (PTCMUT) and asymmetric dyads (PTCASY)
    PTCMUT = mutual_d / total_d
    #PTCASY = asym_d / total_d
    
    
    # count total out_degree connections
    out_degree = connectivity.sum()
    # average out_degree
    av_degree = out_degree / nodes
    # take the sum of squares of the out degree connections per individual (row)
    sum_squares_out = (connectivity.sum(axis=1)**2).sum()
    
    # calculate mutuality index (RHO2) (according to Katz and Powell’s (1955))
    RHO2 = (2*(nodes - 1)**2 * mutual_d - out_degree**2 + sum_squares_out) / (out_degree*(nodes - 1)**2 - out_degree**2 + sum_squares_out)
    
    # determine the local clustering coefficient mean and standard deviation
    clustering_coefficients = []
    for n_node, connections in enumerate(connectivity):
        # the amount of neighbours each node has
        n_neighbours = np.sum(connectivity[n_node])
        # only consider nodes with at least 2 neighbours
        if n_neighbours >= 2:
            # matrix of the nodes that are both neighbours of the node considered
            neighbour_matrix = np.dot(np.transpose([connectivity[n_node]]),[connectivity[n_node]])
            # the amount of connections between neighbours
            neighbour_connections = np.sum(connectivity*neighbour_matrix)
            # the amount of connections between neighbours divided by the possible amount of connections
            clustering_coefficients.append(neighbour_connections / (n_neighbours*(n_neighbours-1)))
    mean_clustering_coefficient = np.mean(clustering_coefficients)
    std_clustering_coefficient = np.std(clustering_coefficients)
    clustering_coefficient = [mean_clustering_coefficient,std_clustering_coefficient]
    
    
    # create homophily index dictionary and lists for the segregation index
    homoph_ind = dict()
    segreg_ind = []
    
    # iterate through different characteristics (sex, race, grade)
    for i in range(characteristics.shape[1]):
        # get different groups of this characteristic in dataset
        characs = sorted(list(set(characteristics[:,i])))
        amount = len(characs)
        
        # counters for individuals per characteristic group and out-group nominations
        charac_count = []
        charac_out = []
        
        # counters for expected and observed cross trait nominations
        exp_cross = 0        
        obs_cross = 0
        
        # iterate through different groups of this characteristic
        for j in range(amount):
            # indicate indices of members this group and save size group
            indices = np.where(characteristics[:,i] == characs[j])[0]
            charac_count.append(len(indices))
            
            # create a submatrix of all nominations from this group and save amount
            submat_trait = connectivity[np.ix_(indices,)]
            charac_out.append(submat_trait.sum())
            
            # create 2 submatrices outgoing connections: 1 to individuals same group and 1 to individuals different group
            submat_same = connectivity[np.ix_(indices,indices)]
            mask = np.ones(connectivity.shape[0], np.bool)
            mask[indices] = 0
            submat_diff = submat_trait[:,mask]
            
            # count amount outgoing connections to same and to different group
            out_same = np.mean(submat_same.sum(axis=1))
            out_diff = np.mean(submat_diff.sum(axis=1))
            
            # add amount of cross trait nominations of this group to total
            obs_cross += submat_diff.sum()
            
            # calculate and save homophily index from this group for this characteristic
            homoph_ind[str(i) + "_" + str(characs[j])] = out_same / (out_same + out_diff)
        
        # calculate expected cross trait nominations (added for all combinations of traits (except with self))
        for row in range(amount):
            for col in range(amount):
                if row == col: pass
                else:
                    exp_cross += charac_out[row] * ((charac_count[row] * charac_count[col]) / (charac_count[row] * (nodes - 1)))
        
        # calculate and save segregation index for this trait
        if exp_cross == 0:
            segreg_ind.append(999)
            if amount == 1:
                warnings.warn("Just one group: no segregation index for this trait possible.")
            else:
                warnings.warn("Expected cross trait nominations is zero: no segregation index for this trait possible.")
        else:
            segreg_ind.append((exp_cross - obs_cross) / exp_cross)
    
    segreg_ind
    
    return av_degree, RCHDEN, RELDEN, PTCMUT, RHO2, clustering_coefficient, homoph_ind, segreg_ind
    

In [266]:
analyse_network(connectivity, charac_short)

(4.10091743119266,
 0.7214576962283384,
 0.41009174311926605,
 0.2808022922636103,
 0.41640636493060856,
 [0.32442072614486406, 0.20200116408940086],
 {'0_1': 0.5807692307692308,
  '0_2': 0.6310160427807486,
  '1_1': 0.9447115384615384,
  '1_3': 0.0,
  '1_5': 0.0,
  '2_10': 0.6122448979591837,
  '2_11': 0.6666666666666666,
  '2_12': 0.8139534883720931,
  '2_7': 0.8321678321678322,
  '2_8': 0.8,
  '2_9': 0.5135135135135136},
 [0.18525111252383983, 0.1111111111111111, 0.704991980753809])